# 순서

1.결론

2.LMS 노드 내용 정리

3.코드

## 1.결론

분산표현을 구현하기 위한 방법 : Tokenizing

1.Token 단위 : 1)word, 2)sub-word(BPE, WPM, SentencePiece), 3)형태소

2.Tokenizing 방법 : BPE, WPM, SentencePice

3.Token to Vector 방법 : word2vec(CBOW, Skip-gram), FastText, ELMo

희소표현 : 1)indices, 2)one-hot vector

## 2.노드 내용 정리


#### 학습 목표

1.분산표현에 대한 직관적 이해를 얻는다.

2.문장 데이터를 정제하는 방법을 배운다.

3.토큰화의 여러 가지 기법들을 배운다.

4.단어 Embedding을 구축하는 방법에 대해 가볍게 맛본다




자연어, 인공어, 프로그래밍 언어
문맥자유문법(context free grammer), 문맥 의존 문법(Context-sensitive grammer)
syntaxnet, google - https://ai.googleblog.com/2016/05/announcing-syntaxnet-worlds-most.html
real world knowledge

1-2.언어모델 - 단어가 출현하게될 확률 모델, 일정한 패턴 => 통계/딥러닝 학습 => 문법/언어관습 패턴

불완전한 대화
- 문장 길이가 너무 짧거나 긴 경우
- 채팅 데이터간 문장 시간간격이 너무 긴 경우
- 바람직하지 않은 문장의 사용 - 욕설, 오타
- 노이즈1) 문장부호 - replace
- 노이즈2) 대소문자
- 노이즈3) 특수문자
정규표현식(re)

1-3.분산표현
분산표현 vs 희소표현
- 희소표현 : 단어를 고차원 벡터로 변환
- 너무 고차원은 불가 : 두 단어의 코사인 유사도를 통한 유사도 구하기

분산표현 : 임베딩 레이어를 사용해 각 단어가 몇 차원의 속성을 가질질 정의
일일이 정의할 수 없는 추상적인 속성을 256 차원 안에 골고루 분산되게 표현

단어사전 구성, 활용의 문제 : 컴퓨터 vs 사람의 인식 차이

1.4.토큰화 : 그녀는? 그녀 + 는?
문장을 어느 단위까지, 어느 레벨까지 자를 것이냐? : 토큰화의 기준
1)공백기반 토큰화 : .split()
2)형태소 기반 토큰화 : 1)은 문제가 많아. - konlpy, khaii, komoran, 꼬꼬마, mecab
성능비교 : https://iostream.tistory.com/144https://iostream.tistory.com/144

사전에 없는 단어의 문제
형태소 기반 토큰화 기법 : 모두 의미를 가지는 단위로 토큰을 생성함.
빈도수 기준 상위 N 개 단어만 사용하고, 나머지는 unk 특수토큰 치환 : 손실
: out of vacabulary 문제 : 어떻게 하냐?


1-5. Wordpiece Model(WPM)
preview, predict : pre- 라는 공동된 접두어 가지고 있음
학습 시, pre+view, pre+dict 로 학습하면 더 잘 되지 않을까?
pre : subword 같이 '한 단어를 여러 개의 subword 의 집합으로 보는 방법이 WPM'

#1)BPE(byte-pair encoding)

1994년 데이터 압축을 위해 생겨남.
데이터에서 가장 많이 등장하는 바이트쌍(byte-pair)를 새로운 단어로 치환하여 압축하는 작업을 반복

2015년 위 bpe 를 토큰화에 적용하자고 제안.
모든 단어를 문자(byte)들의 집합으로 취급
장점1 : 접두어/접미어 의미 캐치할수 있음. 
2 : 처음 등장하는 단어는 문자/알파벳 조합으로 나타내면 oov 문제 해결


lowest : low+est 의 결합으로 표현가능.
큰 데이터도 원하는 크기로 사전을 정의 가능함.

embedding layer 는 [단어의 개수 * 임베딩 차원수] 의 weight 를 생성함
단어의 개수가 줄어들면, 메모리 절약 가능함.
bpe 를 적용하면 단어들을 쪼개기는 쉬우나, 조합하는데 어려움이 있다.

#2)Wordpiece Model(WPM)

1)byte-pair 로 쪼개고, 시작부분에 _ 를 추가함
2)빈도수 기반이 아닌 가능도(likelihood)를 증가시키는 방향으로 문자 쌍을 합침

한국어, 일본어 등 조사/어미의 활용이 높은 모델은 WPM 이 좋은 대안
WPM은 어떤 언어든 무관하게 적용한 language-neutral, general 한 기법

확률, 가능도 : https://jjangjjong.tistory.com/41

Katz's Back-off Model : LM 에서 등장하지 않은 단어에 대해 확률을 할당하는 모델

google bpe github : https://github.com/google/sentencepiece

sentencepiece 상당히 좋으나, '차' 라는 단어가 마시는 차인지 타고다니는 차인지 의미에 대해서는 정보가 없음

1-6)토큰에게 의미를 부여하기
- word2vec, FastText, ELMo

#word2vec - https://wikidocs.net/22660
word2vec 2가지 방식 : 1CBOW, 2,Skip-gram
논리적으로는 CBOW 가 좋아보이나, 성능은 skip-gram 이 우세함

#FastText
word2vec 은 좋은 방법이지만, 연산의 빈부격차가 존재함.
자주 등장하지 않는 단어는 최악의 경우 1번의 연산만으로 random 하게 초기화된 상태로 알고리즘 종료
이를 해결하기 위해 FASTTEXT 는 BPE 와 비슷한 아이디어 적용함.
FastText 는 한 단어를 n-gram 의 집합으로 보고, 단어를 쪼개서 n-gram 에 할당된 embedding 의 평균값을 사용함
https://brunch.co.kr/@learning/7

#ELMo
- the 1st contextualized Word embedding
위 word2vec, FastText 는 고질적인 문제가 있음. 고정적
사과가 먹는사과인지, 용서를 구하는 사과인지?
즉, 단어의 의미를 정하려면, 문장의 context 가 필요하다.
먹음직한 사과 : 먹는 사과
무릎꿇고 사과를 하다 : 용서를 구하는 사과

이런 개념을 Contextualized Word Embedding 이라고 한다.
링크 : https://brunch.co.kr/@learning/12



1-7)마무리
데이터에 sentencepiece 적용 후 word2vec 를 적용해 유사도 비교 해보고 싶다.

오늘 배운 내용 NLP 공부하게되면 계속 나올 것이다.(BPE 등)

## sentence piece

https://github.com/google/sentencepiece

SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.



### Subword NMT(Neural Machine Translation)



## 3.코딩

## 1-3 분산표현

In [2]:
import tensorflow as tf

embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)

## 공백 기반 토큰화

In [3]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

# 형태소 기반 토큰화

한국어 형태소 분석기는 대표적으로 아래 두 가지가 사용됩니다.

KoNLPy: 파이썬 한국어 NLP - KoNLPy 0.4.3 documentation

kakao/khaiii

▲ Khaiii는 Windows를 지원하지 않습니다!

KoNLPy는 내부적으로 5가지의 형태소 분석 Class를 포함하고 있습니다. Khaiii까지 총 6개나 되는 형태소 분석기들은 특수한 문장(띄어쓰기 X / 오탈자) 처리 성능, 속도 측면에서 차이를 보입니다. 천하무적인 것은 (아직은) 없으니, 각 분석기를 직접 테스트해보고 적합한 것을 선택해 사용하면 됩니다.

아래 웹페이지에서 그 실험을 대신해주었네요. 정말 좋은 참고 자료입니다!

한국어 형태소 분석기 성능 비교

분석기	로딩 시간 (초)
khaiii	0.0016
한나눔	0.0001
꼬꼬마	0.0002
KOMORAN	0.9542
Open Korean Text (이하 OKT)	0.0001
mecab	0.0004

mecab : 속도 최우선, 품질 상위권
KOMORAN : 자소 분리 / 오탈자 품질 보장
한나눔, khaiii : 분석 품질 별로
꼬꼬마 : 분석 시간 별로

In [4]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

"""
$ pip install konlpy
$ cd ~/aiffel/text_preprocess
$ git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
$ cd Mecab-ko-for-Google-Colab
$ bash install_mecab-ko_on_colab190912.sh

$ sudo apt update
$ sudo apt install default-jre

khaiii 설치!
위와 같은 초기화 코드가 실행 시 오류가 발생한다면 다음과 같은 설치 과정이 필요합니다. 원활한 설치를 위해 Khaiii 설치 과정을 다음과 같은 스크립트로 제공해 드립니다.

$ sudo apt install cmake   # Khaiii의 빌드 과정을 위해 cmake를 필요로 합니다.
$ pip install torch     # Khaiii는 구동을 위해 파이토치를 필요로 합니다. 
$ cd ~/aiffel/text_preprocess
$ git clone https://github.com/modulabs/khaiii_wrapper.git
$ ./install_khaiii.sh


"""

In [5]:
import khaiii

api = khaiii.KhaiiiApi()
api.open()

ModuleNotFoundError: No module named 'khaiii'

In [8]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt()] #,Khaiii()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))


AttributeError: 'java.lang.String' object has no attribute 'splitlines'

## 1-5 WPM

## BPE

In [16]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 10

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():   # word : l o w, # freq : 5
        
        symbols = word.split() # symbols : l, o, w
        
        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
            #print("get_stats, pairs : ", pairs)
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    print("best : ", best)
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

>> Step 1
best :  ('e', 's')
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
best :  ('es', 't')
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
best :  ('l', 'o')
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
best :  ('lo', 'w')
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
best :  ('n', 'e')
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

>> Step 6
best :  ('ne', 'w')
다음 문자 쌍을 치환: new
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'new est ': 6, 'w i d est ': 3} 

>> Step 7
best :  ('new', 'est')
다음 문자 쌍을 치환: newest
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'newest ': 6, 'w i d est ': 3} 

>> Step 8
best :  ('w', 'i')
다음 문자 쌍을 치환: wi
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'newest ': 6, 'wi d est ': 3} 

>> Step 9
best :  ('wi',

# LMS 종료